<h1 style="color: #492c68;">01 | BASICS</h1>

<h2 style="color: #327a81;">Libraries</h2>

In [1]:
## Basic libraries

import pandas as pd
import ast #
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## Settings

pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

<h2 style="color: #327a81;">Data Read</h2>

In [2]:
data= pd.read_csv("netflix_mood_recommender.csv")

In [3]:
df = data.copy()

<h1 style="color: #492c68;">02 | MOOD RECOMMENDER</h1>

- Let's finally build our mood recommender. We will code a way to ask the user for a kind of mood and we'll show suggestions from our titles

In [4]:
# First let's check a sample our final df

df.sample(2)

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood
7374,s7375,Movie,Madre,Aaron Burns,"Daniela Ramírez, Cristobal Tapia Montt, Aida, ...",Chile,2017-08-26,TV-MA,"International Movies, Thrillers",NaN,95.0,"Pregnant and overwhelmed, Diana hires a careta...","fear, surprise"
4701,s4702,Movie,For Here or to Go?,Rucha Humnabadkar,"Ali Fazal, Melanie Chandra, Rajit Kapoor, Amit...","United States, India",2018-08-15,TV-MA,"Comedies, Dramas, Independent Movies",NaN,105.0,A software engineer must decide if he'd rather...,"disgust, anger"


In [5]:
df.shape

(8807, 13)

<h2 style="color: #327a81;">Mood one hot encoding</h2>

- Let's “hot code” the moods in the dataset so that our recommender works more efficiently.

In [6]:
# First we stack moods. One by one for each title

mood_encoding = df.drop(columns="mood").join(df["mood"].str.split(", ", expand=True).stack().reset_index(level=1, drop=True).rename("mood"))

In [7]:
# Then we apply "get dummies" to encode all mood possibilities

mood_encoding = pd.get_dummies(mood_encoding, columns=["mood"])*1

In [8]:
# Check out the encoding results

mood_encoding.head(2)

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,0,1,0
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,1,0,0


In [9]:
# Filtering the obtained dataframe to show all encoded moods by show id

mood_columns = ["mood_anger", "mood_disgust", "mood_fear", "mood_joy", "mood_sadness", "mood_surprise"]
total_moods = mood_encoding.groupby("show_id")[mood_columns].sum()

# And merge the og dataframe with the encoded moods by show id

df_moods = df.merge(total_moods, on="show_id", how="inner")
df_moods

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...","sadness, joy",0,0,0,1,1,0
1,s2,TV Show,Blood & Water,Not Provided,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",2.0,NaN,"After crossing paths at a party, a Cape Town t...","disgust, fear",0,1,1,0,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Not Provided,2021-09-24,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",1.0,NaN,To protect his family from a powerful drug lor...,"anger, disgust",1,1,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,Not Provided,Not Provided,Not Provided,2021-09-24,TV-MA,"Docuseries, Reality TV",1.0,NaN,"Feuds, flirtations and toilet talk go down amo...","disgust, joy",0,1,0,1,0,0
4,s5,TV Show,Kota Factory,Not Provided,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",2.0,NaN,In a city of coaching centers known to train I...,"disgust, fear",0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,R,"Cult Movies, Dramas, Thrillers",NaN,158.0,"A political cartoonist, a crime reporter and a...","disgust, fear",0,1,1,0,0,0
8803,s8804,TV Show,Zombie Dumb,Not Provided,Not Provided,Not Provided,2019-07-01,TV-Y7,"Kids' TV, Korean TV Shows, TV Comedies",2.0,NaN,"While living alone in a spooky town, a young g...","joy, fear",0,0,1,1,0,0
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,R,"Comedies, Horror Movies",NaN,88.0,Looking to survive in a world taken over by zo...,"fear, disgust",0,1,1,0,0,0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,PG,"Children & Family Movies, Comedies",NaN,88.0,"Dragged from civilian life, a former superhero...","disgust, fear",0,1,1,0,0,0


<h2 style="color: #327a81;">Coding the recommender</h2>

<h3 style="color: #60b671;">Functions for the recommender</h3>

- Next, we will develop the functions by steps that, at the end, will constitute the main function that will recommend a title according to the user's inputs.

In [10]:
# In type_selection, user could select between movies and tv shows

def type_selection():
    while True:
        choice = input("Do you want to watch a Movie or a Tv Show?: ").strip().lower()
        if choice == "movie":
            type_mood = df_moods[df_moods["type"] == "Movie"]
            break
        elif choice == "tv show":
            type_mood =  df_moods[df_moods["type"] == "TV Show"]
            break
        else:
            print(f"{choice} is an invalid choice. Please choose either Movie or TV Show.")
            
    return type_mood

In [11]:
# mood_selection will ask for the user's mood. Could be 1 or 2 moods

def mood_selection():
    
    moods = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
    
    while True:
        mood1 = input(f"Select a mood according to what you want to see ({', '.join(moods)}): ").strip().lower()
        if mood1 not in moods:
            print(f"{mood1} is not a valid choice. Please choose from ({', '.join(moods)}): ")
        else:
            break
    while True:
        mood2 = input(f"Select a second mood (this is optional, press enter to skip): ").strip().lower()
        if mood2 and mood2 not in moods:
            print(f"{mood2} is not a valid choice. Please choose from ({', '.join(moods)}) or press enter to skip: ")
        else:
            break
    
    return mood1, mood2

In [12]:
# mood_recommendation will filter the dataframe grouped by type_selection and will have a new dataframe filtered by mood_selection's mood

def mood_recommendation(type_mood, mood1, mood2):
    
    mood1_column = f"mood_{mood1}"
    if mood2:
        mood2_column = f"mood_{mood2}"
        recommended = type_mood[(type_mood[mood1_column] == 1) & (type_mood[mood2_column] == 1)]
    else:
        recommended = type_mood[type_mood[mood1_column] == 1]
    return recommended

<h3 style="color: #60b671;">Recommender main function</h3>

- After defining the previous functions, we can build the main function that recommends a title 

In [13]:
# We define the function ordering the inside functions and printing the relevant information that answers to user's inputs

def netflix_recommender():

    type_mood = type_selection()
    mood1, mood2 = mood_selection()
    recommended = mood_recommendation(type_mood, mood1, mood2)
    item_suggested = recommended.sample(1)
    print(f"")
    print(f"Title: {item_suggested['title'].values[0]}")
    print(f"Sentiment Mood: {item_suggested['mood'].values[0]}")
    print(f"Categorie: {item_suggested['listed_in'].values[0]}")
    print(f"Director: {item_suggested['director'].values[0]}")
    print(f"Cast: {item_suggested['cast'].values[0]}")
    print(f"Country: {item_suggested['country'].values[0]}")
    print(f"Rating: {item_suggested['rating'].values[0]}")

    if item_suggested["type"].values[0] == "Movie":
        print(f"Movie Length: {int(item_suggested['movie_lenght'].values[0])} min")
    elif item_suggested["type"].values[0] == "TV Show":
        print(f"Number of Seasons: {item_suggested['n_seasons'].values[0]}")
    
    print(f"Description: {item_suggested['description'].values[0]}")
    print(f"")  

In [14]:
# And finally we write a block that works as the recommender

while True:
    netflix_recommender()
    new_recommendation = input("Would you like another recommendation? (yes/no): ").strip().lower()
    if new_recommendation == "no":
        print("Hope you like your selection! Enjoy!")
        break
        

Do you want to watch a Movie or a Tv Show?:  tv show
Select a mood according to what you want to see (anger, disgust, fear, joy, sadness, surprise):  anger
Select a second mood (this is optional, press enter to skip):  fear



Title: A Very Secret Service
Sentiment Mood: fear, anger
Categorie: International TV Shows, TV Comedies
Director: Not Provided
Cast: Hugo Becker, Wilfred Benaïche, Christophe Kourotchkine, Karim Barras, Bruno Paviot, Jean-Édouard Bodziak, Mathilde Warnier, Joséphine de La Baume, Marie-Julie Baup, Antoine Gouy, Julie Farenc
Country: France
Rating: TV-MA
Number of Seasons: 2.0
Description: At the height of the Cold War in 1960, André Merlaux joins the French Secret Service and contends with enemies both foreign and bureaucratic.



Would you like another recommendation? (yes/no):  no


Hope you like your selection! Enjoy!


<h1 style="color: #492c68;">01 | EXPORTING CSV</h1>

- Next step is create a streamlit app for this recommender, so let's export a new csv that includes the hot encoding columns

In [15]:
df_moods.to_csv("df_moods.csv", index=False)